# DSSP

Annotation of secondary structure, solvent accessibility and more.

Biopython DSSP module documentation: https://biopython.org/docs/1.75/api/Bio.PDB.DSSP.html

DSSP:

- Webserver: https://www3.cmbi.umcn.nl/xssp/

- Documentation: https://swift.cmbi.umcn.nl/gv/dssp/

- Download: https://github.com/cmbi/dssp

## 1. From PDB file

! Installed DSSP program is required.

If you have DSSP installed locally, you can obtain DSSP annotation using `dssp_dict_from_pdb_file()` for the given PDB file.

In [1]:
!wget https://files.rcsb.org/download/1A1L.pdb

--2022-05-06 01:07:38--  https://files.rcsb.org/download/1A1L.pdb
Resolving files.rcsb.org (files.rcsb.org)... 128.6.158.49
Connecting to files.rcsb.org (files.rcsb.org)|128.6.158.49|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: '1A1L.pdb'

1A1L.pdb                [  <=>               ] 133.76K   377KB/s    in 0.4s    

2022-05-06 01:07:39 (377 KB/s) - '1A1L.pdb' saved [136971]



In [2]:
from Bio.PDB.DSSP import *

In [32]:
file = '1A1L.pdb'
# the function returns a tuple where the first and the only element is a dictionary
dssp = dssp_dict_from_pdb_file(file)[0]

### DSSP object

A dictionary with the information for every residue. Both keys and values are tuples.

**Elements of keys:**

0 - chain identifier \
1 - tuple, where the second element is residue number 

**Elements of values:**

0 - Amino acid \
1 - Secondary structure \
2 - Relative ASA \
3 - Phi \
4 - Psi \
5 - Residue index \
6 - NH–>O_1_relidx \
7 - NH–>O_1_energy \
8 - O–>NH_1_relidx \
9 - O–>NH_1_energy \
10 - NH–>O_2_relidx \
11 - NH–>O_2_energy \
12 - O–>NH_2_relidx \
13 - O–>NH_2_energy

### Secondary structure


H - Alpha helix (4-12) \
B - Isolated beta-bridge residue \
E - Strand \
G - 3-10 helix \
I - Pi helix \
T - Turn \
S - Bend \
&ndash; - None 

In [5]:
for i in dssp:
    print(i)
    break

('A', (' ', 103, ' '))


In [7]:
for keys, values in dssp.items():
    print(keys, values)
    break

('A', (' ', 103, ' ')) ('R', '-', 186, 360.0, 77.2, 1, 0, 0.0, 14, -0.2, 0, 0.0, 15, -0.1)


In [11]:
# print residues involved in a strand

for keys, values in dssp.items():
    chain = keys[0]
    res = keys[1][1]
    aa = values[0]
    ss = values[1]
    if ss == 'E':
        print('residue', aa, res, 'chain', chain)

residue Y 105 chain A
residue A 106 chain A
residue R 115 chain A
residue F 116 chain A
residue F 135 chain A
residue Q 136 chain A
residue N 143 chain A
residue F 144 chain A
residue F 163 chain A
residue A 164 chain A
residue K 171 chain A
residue F 172 chain A


##  2. From DSSP file

You can generate DSSP file or download it from DSSP database and parse it using `make_dssp_dict()`.

Documentation: https://biopython.org/docs/1.75/api/Bio.PDB.DSSP.html#Bio.PDB.DSSP.make_dssp_dict

In [12]:
!mkdssp -i 1A1L.pdb -o 1A1L.dssp

In [36]:
dssp_dict = make_dssp_dict('1A1L.dssp')[0]
for i in dssp_dict.items():
    print(i)
    break

(('A', (' ', 103, ' ')), ('R', '-', 186, 360.0, 77.2, 1, 0, 0.0, 14, -0.2, 0, 0.0, 15, -0.1))


<img src="dssp.png" width='900px'>

## 3. From structure object

! Installed DSSP program is required.

Another way to parse DSSP is to, firstly, parse the PDB structure, and then generate secondary structure annotation using `DSSP()`. In this case the output will be a dictionary, but, in contrast to the previous outputs, the residue index will be the first element (not sixth) and instead of absolute values of accessible surface area there will be values of relative ASA. By default the `acc_array` parameter is `"Sander"` - the max ASA values from Sander & Rost (1994). See other options: https://biopython.org/docs/1.75/api/Bio.PDB.DSSP.html#Bio.PDB.DSSP.DSSP.

In [37]:
from Bio.PDB import PDBParser
p = PDBParser()

structure = p.get_structure("1A1L", "1A1L.pdb")
model = structure[0]

/usr/local/lib64/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 1574.
  PDBConstructionWarning,
/usr/local/lib64/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 1595.
  PDBConstructionWarning,
/usr/local/lib64/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 1609.
  PDBConstructionWarning,


In [38]:
dssp = DSSP(model, "1A1L.pdb", acc_array='Miller')

In [39]:
for i in dssp:
    print(i)
    break

(1, 'R', '-', 0.7717842323651453, 360.0, 77.2, 0, 0.0, 14, -0.2, 0, 0.0, 15, -0.1)


Print the proportions of exposed and buried residues:

In [40]:
exposed, buried = 0, 0

for i in dssp:
    if i[3] >= 0.25:
        exposed+=1
    elif i[3] < 0.25:
        buried+=1
        
e = exposed/len(dssp)
b = buried/len(dssp)

In [41]:
print("Exposed: {0:.0%}".format(e))
print("Buried: {0:.0%}".format(b))

Exposed: 79%
Buried: 21%
